# Import & Setup

In [21]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [22]:
import pandas as pd
import numpy as np
from myfunctions import clean_path

In [23]:
df = pd.read_csv(filepath_or_buffer=clean_path(r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/Policing Thesis/Modified Dataset - 2021.csv"),
                 index_col='Stop ID')

# CrossTabs

## Accidents

### Can one stop get multiple accident rows?

Yes

In [24]:
df['Accident'].sum()

1370

In [27]:
df[df['Accident']==1].index.unique()

Int64Index([   19,    22,   140,   162,   205,   206,   213,   254,   318,
              321,
            ...
            13092, 13094, 13095, 13099, 13216, 13220, 13252, 13268,  9384,
            13358],
           dtype='int64', name='Stop ID', length=484)

### Get # Accidents (no duplicates)

In [67]:
def get_accidents_crosstab(dataframe:pd.DataFrame) -> pd.Series:
    
    accident_col_with_ID = df.query('Accident==1')['Race'].reset_index()
    
    grouped_accident_col = accident_col_with_ID.groupby(by='Stop ID').agg(set)
    
    grouped_accident_col['Race'] = grouped_accident_col['Race'].apply(list).apply(lambda x: x[0])
    
    return grouped_accident_col.value_counts().sort_index() / len(grouped_accident_col)
    
accidents_crosstab = get_accidents_crosstab(df)
accidents_crosstab.name = 'Accidents'
print(accidents_crosstab)

Race    
ASIAN       0.045455
BLACK       0.285124
HISPANIC    0.322314
OTHER       0.039256
WHITE       0.307851
Name: Accidents, dtype: float64


In [59]:
df.query('Accident==1')['Accident'].reset_index()

,Stop ID,Accident
0,19,1
1,22,1
2,140,1
3,162,1
4,162,1
...,...,...
1365,13220,1
1366,13252,1
1367,13268,1
1368,9384,1


In [ ]:
# get all rows with accidents
# for each unique ID, get one race value
# add to list
# end up with list of [race of accident #1, race of accident #2, ..., race of accident #n]
# get frequency of values